In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [ ]:
df = pd.read_csv("preprocessing\dataset_plantae_rj.csv", sep=";", encoding="latin1", dtype=str)
df_sem_duplicatas = df.drop_duplicates(subset=['nome_cientifico', 'latitude', 'longitude'])

print(f"Arquivo sem duplicatas salvo com {len(df_sem_duplicatas)} linhas.")
print(f"Arquivo com duplicatas tinha {len(df)} linhas.")
print(f"Removidas {len(df) - len(df_sem_duplicatas)} linhas duplicadas.")

In [ ]:
df_sem_duplicatas.to_csv("preprocessing\dataset_plantae_rj_sem_duplicatas.csv", index=False, sep=';', encoding='latin1')

In [ ]:
contagem_especies = df["nome_cientifico"].value_counts()
contagem_especies_unique = df_sem_duplicatas["nome_cientifico"].value_counts()

In [ ]:
contagem_especies.describe()

In [ ]:
contagem_especies_unique.describe()

In [ ]:
plt.figure(figsize=(10,6))
contagem_especies.plot(kind="hist", bins=50, edgecolor='black')
plt.title("Distribuição da quantidade de registros por espécie")
plt.xlabel("Número de registros por espécie")
plt.ylabel("Frequência")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
contagem_especies_unique.plot(kind="hist", bins=50, edgecolor='black')
plt.title("Distribuição da quantidade de registros por espécie")
plt.xlabel("Número de registros por espécie")
plt.ylabel("Frequência")
plt.grid(True)
plt.show()

In [ ]:
limite = 30

especies_filtradas = contagem_especies_unique[contagem_especies_unique >= limite].index
df_plants = df[df['nome_cientifico'].isin(especies_filtradas)]


In [ ]:
df_plants.to_csv("dataset_plantae_rj_min_30.csv", index=False, sep=';', encoding='latin1')
contagem_especies_final= df_plants["nome_cientifico"].value_counts()
# Informações úteis
print("Registros originais:", len(df_sem_duplicatas))
print("Registros após filtragem (mínimo 30):", len(df_plants))
print("Registros removidos:", len(df_sem_duplicatas) - len(df_plants))

In [ ]:
plt.figure(figsize=(10,6))
contagem_especies_final.plot(kind="hist", bins=50, edgecolor='black')
plt.title("Distribuição da quantidade de registros por espécie")
plt.xlabel("Número de registros por espécie")
plt.ylabel("Frequência")
plt.grid(True)
plt.show()

In [ ]:
df_climasolotemp_rj = pd.read_csv("dataset_climasolotemp_rj.csv", sep=";", encoding="latin1", dtype=str)


In [ ]:
df_plants['latitude'] = df_plants['latitude'].astype(str).str.replace(',', '.')
df_plants['longitude'] = df_plants['longitude'].astype(str).str.replace(',', '.')



In [ ]:
df_plants[['latitude', 'longitude']] = df_plants[['latitude', 'longitude']].replace('', pd.NA)
df_plants = df_plants.dropna(subset=['latitude', 'longitude'])

In [ ]:
df_plants['latitude'] = df_plants['latitude'].astype(float)
df_plants['longitude'] = df_plants['longitude'].astype(float)

In [ ]:
df_plants.columns
df_plants['latitude']

In [ ]:
df_plants['longitude']

In [ ]:
# Supondo que df_plants e df_clima já estejam com latitude/longitude como float
# 1. Extrai as coordenadas como arrays NumPy
plant_coords = df_plants[['latitude', 'longitude']].to_numpy()
climasolotemp_coords = df_climasolotemp_rj[['latitude', 'longitude']].to_numpy()

In [ ]:
# 2. Treina o modelo de vizinho mais próximo (mais rápido com 'ball_tree')
nn = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
nn.fit(climasolotemp_coords)

In [ ]:
# 3. Para cada planta, encontra o ponto de clima mais próximo
distances, indices = nn.kneighbors(plant_coords)

In [ ]:
# 4. Pega as linhas correspondentes do df_clima
df_climasolotemp_matched = df_climasolotemp_rj.iloc[indices.flatten()].reset_index(drop=True)

In [ ]:
# 5. Remove lat/lon duplicadas do clima para evitar redundância
df_climasolotemp_matched = df_climasolotemp_matched.drop(columns=['latitude', 'longitude'])


In [ ]:
# 6. Reseta o índice do df_plants também para garantir alinhamento
df_plants = df_plants.reset_index(drop=True)

In [ ]:
# 9. Concatena os dois DataFrames (plantas + climasolotemp)
df_resultado = pd.concat([df_plants, df_climasolotemp_matched], axis=1)

In [ ]:
# 10. Filtra por distância máxima aceitável (ex: 0.05 graus ≈ 5km)
dist_max = 0.05
df_resultado['distancia'] = distances.flatten()
df_resultado = df_resultado[df_resultado['distancia'] <= dist_max].drop(columns='distancia')

In [ ]:
# 11. Salva o resultado final
df_resultado.to_csv('dataset_cruzado2.csv', index=False, sep=';', encoding='latin1')